In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from aging.plotting import format_plots, PlotConfig, save_factory, figure, legend, format_pizza_plots, COLORMAPS
from collections import Counter
from matplotlib.lines import Line2D
from aging.organization.dataframes import load_male_long_df, load_female_long_df, DF_PATHS
from tqdm import tqdm

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import colorcet as cc
from operator import add
from tqdm.auto import tqdm
from functools import reduce
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from scipy.spatial.distance import squareform, pdist
from sklearn.metrics import silhouette_score
from aging.plotting import format_plots, figure, save_factory, PlotConfig, legend, add_identity
from sklearn.cross_decomposition import PLSRegression

In [ ]:
def mm_norm_col(column):
    return (column - column.min()) / (column.max() - column.min())

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

# prepare colors for plot
colors = ['#c7eae5','#008C8D']
import matplotlib
cmap = LinearSegmentedColormap.from_list("custom_purples", colors, N=256)
matplotlib.cm.register_cmap("dana", cmap)
phasec_m = sns.color_palette("dana", n_colors=27)

# prepare colors for plot
colors = ['#fee6ce','#d94801']
import matplotlib
cmap = LinearSegmentedColormap.from_list("custom_purples", colors, N=256)
matplotlib.cm.register_cmap("dana_f", cmap)
phasec_f = sns.color_palette("dana_f", n_colors=27)

import colorcet as cc
fmap = cc.glasbey[0:20]
mmap = cc.glasbey[20:40]

In [ ]:
cmm = COLORMAPS.ont_male
cmf = COLORMAPS.ont_female
m_df = load_male_long_df(average_weeks=True, merge_size=False, merge_ages=True, df_path=DF_PATHS.usage_male).groupby(['age','mouse']).mean()
f_df = load_female_long_df(average_weeks=True, merge_size=False, filter_female=True, merge_ages=True, df_path=DF_PATHS.usage_female).groupby(['age','mouse']).mean()

## normalized

In [ ]:
# avarage per age or session
avg_f_df = f_df.groupby(['age']).mean()

# normalize the data
f_norm = mm_norm_col(f_df)
avg_f_norm = mm_norm_col(avg_f_df)

# avarage per age or session
avg_m_df = m_df.groupby(['age']).mean()

# normalize the data
m_norm = mm_norm_col(m_df)
avg_m_norm = mm_norm_col(avg_m_df)

In [ ]:
format_plots()

## umaps

In [ ]:
import umap
df=m_norm.copy()
reducer = umap.UMAP(random_state=10,n_neighbors=200)
umap_result = reducer.fit_transform(df)

pcs = pd.DataFrame(umap_result, columns=['dim1','dim2'])
pcs['age'] = df.index.get_level_values('age')
pcs['mouse'] = df.index.get_level_values('mouse')
age_pc = pcs.drop('mouse', axis=1).groupby('age').mean()
# Plot
fig, ax = plt.subplots()
fig.set_size_inches(1.3, 1.3)
im=sns.scatterplot(data=pcs, x='dim1',y='dim2',hue='age',palette=cmm, legend=False, s=3, edgecolor='none')
sns.despine()

In [ ]:
c = PlotConfig()
fig.savefig(c.dana_save_path / "fig3"/ 'UMAP_male_age_longv2_n200.pdf')

In [ ]:
import pandas as pd
import umap
import matplotlib.pyplot as plt
import seaborn as sns
# Assuming df contains your data
# Perform UMAP dimensionality reduction
df = m_norm.copy()
reducer = umap.UMAP(random_state=0, n_neighbors=30)
umap_result = reducer.fit_transform(df)

pcs = pd.DataFrame(umap_result, columns=['dim1','dim2'])
pcs['age'] = df.index.get_level_values('age')
pcs['mouse'] = df.index.get_level_values('mouse')

# Plot
fig, ax = plt.subplots()
fig.set_size_inches(1.7, 1.3)

# Scatter plot
im = ax.scatter(
    x=pcs['dim1'], y=pcs['dim2'], 
    c=pcs['age'], 
    cmap=cmm,
    s=3,
)

# Add colorbar
cb = fig.colorbar(im, ax=ax)
tick_values = [5,15,25,35,45]
# Customize the plot
sns.despine()
#plt.title('UMAP Plot of Data')
plt.xlabel('dim1')
plt.ylabel('dim2')
plt.legend()
plt.show()

In [ ]:
# Plot
fig, ax = plt.subplots()
fig.set_size_inches(1.3, 1.3)
im=sns.scatterplot(data=pcs, x='dim1',y='dim2',hue='age',palette=cmm, legend=False, s=3, edgecolor='none')
sns.despine()

In [ ]:
c = PlotConfig()
fig.savefig(c.dana_save_path / "fig3"/ 'UMAP_male_age_longv2_n30.pdf')

In [ ]:
# Plot
fig, ax = plt.subplots()
fig.set_size_inches(1.3, 1.3)
im=sns.scatterplot(data=pcs, x='dim1',y='dim2',hue='mouse',palette=mmap, legend=False, s=3, edgecolor='none')
#cb = fig.colorbar(im, ax=ax)
#plt.title('UMAP Plot of Data males')
sns.despine()

In [ ]:
c = PlotConfig()
fig.savefig(c.dana_save_path / "fig3"/ 'UMAP_male_indv_longv2_n30.pdf')

In [ ]:
# Create a dictionary to map mouse IDs to colors
mouse_colors = dict(zip(pcs['mouse'].unique(), mmap))

# Set the size of the plot
fig, ax = plt.subplots()
fig.set_size_inches(1.3, 1.3)

# Ensure that 'mouse' column is treated as a category for better handling
pcs['mouse'] = pcs['mouse'].astype('category')

# Apply a rolling window to smooth the data and plot lines for each mouse
for mouse_id in pcs['mouse'].unique():
    mouse_data = pcs[pcs['mouse'] == mouse_id]
    mouse_data = mouse_data.sort_values(by='age')  # Sort by 'age' for rolling window
    mouse_data['dim1_smooth'] = mouse_data['dim1'].rolling(window=2, min_periods=1).mean()
    mouse_data['dim2_smooth'] = mouse_data['dim2'].rolling(window=2, min_periods=1).mean()
    ax.plot(mouse_data['dim1_smooth'], mouse_data['dim2_smooth'], label=mouse_id, color=mouse_colors[mouse_id], alpha=0.7, linewidth=1)

# Customize the plot
sns.despine()
plt.xlabel('dim1')
plt.ylabel('dim2')
plt.legend(title='Mouse', bbox_to_anchor=(1.05, 1), loc='upper left')

# Show the plot
plt.show()

In [ ]:
c = PlotConfig()
fig.savefig(c.dana_save_path / "fig3"/ 'UMAP_male_indv_longv2_n30_traj.pdf')

In [ ]:
# Create a dictionary to map mouse IDs to colors
mouse_colors = dict(zip(pcs['mouse'].unique(), mmap))

# Set the size of the plot
fig, ax = plt.subplots()
fig.set_size_inches(1.3, 1.3)

# Ensure that 'mouse' column is treated as a category for better handling
pcs['mouse'] = pcs['mouse'].astype('category')

# Apply a rolling window to smooth the data and plot lines for each mouse
for mouse_id in pcs['mouse'].unique():
    mouse_data = pcs[pcs['mouse'] == mouse_id]
    mouse_data = mouse_data.sort_values(by='age')  # Sort by 'age' for rolling window
    ax.plot(mouse_data['dim1'], mouse_data['dim2'], label=mouse_id, color=mouse_colors[mouse_id], alpha=0.7, linewidth=1)

# Customize the plot
sns.despine()
plt.xlabel('dim1')
plt.ylabel('dim2')
plt.legend(title='Mouse', bbox_to_anchor=(1.05, 1), loc='upper left')

# Show the plot
plt.show()

In [ ]:
c = PlotConfig()
fig.savefig(c.dana_save_path / "fig3"/ 'UMAP_male_indv_longv2_n30_traj_unsmooth.pdf')

In [ ]:
# Define the colormap for ages
import matplotlib.colors as mcolors
norm = mcolors.Normalize(vmin=pcs['age'].min(), vmax=pcs['age'].max())
cmap = cmm
import matplotlib.cm as cm

# Set the size of the plot
fig, ax = plt.subplots()
fig.set_size_inches(1.3, 1.3)

# Ensure that 'mouse' column is treated as a category for better handling
pcs['mouse'] = pcs['mouse'].astype('category')

# Apply a rolling window to smooth the data
smoothed_data = []

for mouse_id in pcs['mouse'].unique():
    mouse_data = pcs[pcs['mouse'] == mouse_id]
    mouse_data = mouse_data.sort_values(by='age')  # Sort by 'age' for rolling window
    mouse_data['dim1_smooth'] = mouse_data['dim1'].rolling(window=2, min_periods=1).mean()
    mouse_data['dim2_smooth'] = mouse_data['dim2'].rolling(window=2, min_periods=1).mean()
    smoothed_data.append(mouse_data)

# Concatenate all smoothed data
smoothed_data = pd.concat(smoothed_data)

# Plot each line segment with gradient colors based on age
for mouse_id in smoothed_data['mouse'].unique():
    mouse_data = smoothed_data[smoothed_data['mouse'] == mouse_id]
    points = mouse_data[['dim1_smooth', 'dim2_smooth']].values
    ages = mouse_data['age'].values
    for i in range(len(points) - 1):
        segment = points[i:i+2]
        age_norm = norm(ages[i])
        color = cmap(age_norm)
        ax.plot(segment[:, 0], segment[:, 1], color=color, linewidth=1, alpha=0.7)

# Customize the plot
sns.despine()
plt.xlabel('dim1')
plt.ylabel('dim2')
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
#cbar = plt.colorbar(sm, ax=ax)
#cbar.set_label('Age')

# Show the plot
plt.show()

In [ ]:
c = PlotConfig()
fig.savefig(c.dana_save_path / "fig3"/ 'UMAP_male_age_longv2_n30_traj.pdf')

In [ ]:
## for females:

In [ ]:
import umap
df=f_norm.copy()
reducer = umap.UMAP(random_state=10,n_neighbors=200)
umap_result = reducer.fit_transform(df)

pcs = pd.DataFrame(umap_result, columns=['dim1','dim2'])
pcs['age'] = df.index.get_level_values('age')
pcs['mouse'] = df.index.get_level_values('mouse')
age_pc = pcs.drop('mouse', axis=1).groupby('age').mean()
# Plot
fig, ax = plt.subplots()
fig.set_size_inches(1.3, 1.3)
im=sns.scatterplot(data=pcs, x='dim1',y='dim2',hue='age',palette=cmf, legend=False, s=3, edgecolor='none')
sns.despine()

In [ ]:
c = PlotConfig()
fig.savefig(c.dana_save_path / "fig3"/ 'UMAP_female_age_n200.pdf')

In [ ]:
import pandas as pd
import umap
import matplotlib.pyplot as plt
import seaborn as sns
# Assuming df contains your data
# Perform UMAP dimensionality reduction
df=f_norm.copy()
reducer = umap.UMAP(random_state=10,n_neighbors=30)
umap_result = reducer.fit_transform(df)

pcs = pd.DataFrame(umap_result, columns=['dim1','dim2'])
pcs['age'] = df.index.get_level_values('age')
pcs['mouse'] = df.index.get_level_values('mouse')
age_pc = pcs.drop('mouse', axis=1).groupby('age').mean()

# Plot
fig, ax = plt.subplots()
fig.set_size_inches(1.7, 1.3)

# Scatter plot
im = ax.scatter(
    x=pcs['dim1'], y=pcs['dim2'], 
    c=pcs['age'], 
    cmap=cmf,
    s=3,
    linewidth =0.25,
)

# Add colorbar
cb = fig.colorbar(im, ax=ax)
tick_values = [5,15,25,35,45]

# Customize the plot
sns.despine()
#plt.title('UMAP Plot of Data females')
plt.xlabel('dim1')
plt.ylabel('dim2')
plt.legend()
plt.show()

In [ ]:
# Plot
fig, ax = plt.subplots()
fig.set_size_inches(1.3, 1.3)
im=sns.scatterplot(data=pcs, x='dim1',y='dim2',hue='age',palette=cmf, legend=False, s=3, edgecolor='none')
sns.despine()

In [ ]:
c = PlotConfig()
fig.savefig(c.dana_save_path / "fig3"/ 'UMAP_female_age_longv2.pdf')

In [ ]:
# Plot
fig, ax = plt.subplots()
fig.set_size_inches(1.3, 1.3)
im=sns.scatterplot(data=pcs, x='dim1',y='dim2',hue='mouse',palette=fmap, legend=False, s=3, edgecolor='none')
#cb = fig.colorbar(im, ax=ax)
#plt.title('UMAP Plot of Data females')

sns.despine()

In [ ]:
c = PlotConfig()
fig.savefig(c.dana_save_path / "fig3"/ 'UMAP_female_indv_longv2.pdf')

In [ ]:
# Create a dictionary to map mouse IDs to colors
mouse_colors = dict(zip(pcs['mouse'].unique(), fmap))

# Set the size of the plot
fig, ax = plt.subplots()
fig.set_size_inches(1.3, 1.3)

# Ensure that 'mouse' column is treated as a category for better handling
pcs['mouse'] = pcs['mouse'].astype('category')

# Apply a rolling window to smooth the data and plot lines for each mouse
for mouse_id in pcs['mouse'].unique():
    mouse_data = pcs[pcs['mouse'] == mouse_id]
    mouse_data = mouse_data.sort_values(by='age')  # Sort by 'age' for rolling window
    mouse_data['dim1_smooth'] = mouse_data['dim1'].rolling(window=2, min_periods=1).mean()
    mouse_data['dim2_smooth'] = mouse_data['dim2'].rolling(window=2, min_periods=1).mean()
    ax.plot(mouse_data['dim1_smooth'], mouse_data['dim2_smooth'], label=mouse_id, color=mouse_colors[mouse_id], alpha=0.7, linewidth=1)

# Customize the plot
sns.despine()
plt.xlabel('dim1')
plt.ylabel('dim2')
plt.legend(title='Mouse', bbox_to_anchor=(1.05, 1), loc='upper left')

# Show the plot
plt.show()

In [ ]:
c = PlotConfig()
fig.savefig(c.dana_save_path / "fig3"/ 'UMAP_female_indv_longv2_traj.pdf')

In [ ]:
# Create a dictionary to map mouse IDs to colors
mouse_colors = dict(zip(pcs['mouse'].unique(), fmap))

# Set the size of the plot
fig, ax = plt.subplots()
fig.set_size_inches(1.3, 1.3)

# Ensure that 'mouse' column is treated as a category for better handling
pcs['mouse'] = pcs['mouse'].astype('category')

# Apply a rolling window to smooth the data and plot lines for each mouse
for mouse_id in pcs['mouse'].unique():
    mouse_data = pcs[pcs['mouse'] == mouse_id]
    mouse_data = mouse_data.sort_values(by='age')  # Sort by 'age' for rolling window
    ax.plot(mouse_data['dim1'], mouse_data['dim2'], label=mouse_id, color=mouse_colors[mouse_id], alpha=0.7, linewidth=1)

# Customize the plot
sns.despine()
plt.xlabel('dim1')
plt.ylabel('dim2')
plt.legend(title='Mouse', bbox_to_anchor=(1.05, 1), loc='upper left')

# Show the plot
plt.show()

In [ ]:
c = PlotConfig()
fig.savefig(c.dana_save_path / "fig3"/ 'UMAP_female_indv_longv2_traj_unsmooth.pdf')

In [ ]:
# Define the colormap for ages
import matplotlib.colors as mcolors
norm = mcolors.Normalize(vmin=pcs['age'].min(), vmax=pcs['age'].max())
cmap = cmf
import matplotlib.cm as cm

# Set the size of the plot
fig, ax = plt.subplots()
fig.set_size_inches(1.3, 1.3)

# Ensure that 'mouse' column is treated as a category for better handling
pcs['mouse'] = pcs['mouse'].astype('category')

# Apply a rolling window to smooth the data
smoothed_data = []

for mouse_id in pcs['mouse'].unique():
    mouse_data = pcs[pcs['mouse'] == mouse_id]
    mouse_data = mouse_data.sort_values(by='age')  # Sort by 'age' for rolling window
    mouse_data['dim1_smooth'] = mouse_data['dim1'].rolling(window=2, min_periods=1).mean()
    mouse_data['dim2_smooth'] = mouse_data['dim2'].rolling(window=2, min_periods=1).mean()
    smoothed_data.append(mouse_data)

# Concatenate all smoothed data
smoothed_data = pd.concat(smoothed_data)

# Plot each line segment with gradient colors based on age
for mouse_id in smoothed_data['mouse'].unique():
    mouse_data = smoothed_data[smoothed_data['mouse'] == mouse_id]
    points = mouse_data[['dim1_smooth', 'dim2_smooth']].values
    ages = mouse_data['age'].values
    for i in range(len(points) - 1):
        segment = points[i:i+2]
        age_norm = norm(ages[i])
        color = cmap(age_norm)
        ax.plot(segment[:, 0], segment[:, 1], color=color, linewidth=1, alpha=0.7)

# Customize the plot
sns.despine()
plt.xlabel('dim1')
plt.ylabel('dim2')
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
#cbar = plt.colorbar(sm, ax=ax)
#cbar.set_label('Age')

# Show the plot
plt.show()

In [ ]:
c = PlotConfig()
fig.savefig(c.dana_save_path / "fig3"/ 'UMAP_female_age_longv2_traj.pdf')